In [1]:
import pyedflib
import numpy as np
import os

In [3]:
current_dir = os.getcwd()
print(current_dir)
subject = "S026"
sub_edf = "S026R01.edf"
file_name = os.path.join(current_dir, subject, sub_edf)
f = pyedflib.EdfReader(file_name)
n = f.signals_in_file
signal_labels = f.getSignalLabels()
sigbufs = np.zeros((n, f.getNSamples()[0]))
for i in np.arange(n):
    sigbufs[i, :] = f.readSignal(i)

/home/giulia/corsi/bio/project/BigBrain


In [4]:
print("\nlibrary version: %s" % pyedflib.version.version)

print("\ngeneral header:\n")

print("edfsignals: %i" % f.signals_in_file)
print("file duration: %i seconds" % f.file_duration)
print("startdate: %i-%i-%i" % (f.getStartdatetime().day,f.getStartdatetime().month,f.getStartdatetime().year))
print("starttime: %i:%02i:%02i" % (f.getStartdatetime().hour,f.getStartdatetime().minute,f.getStartdatetime().second))
print("patientcode: %s" % f.getPatientCode())
print("gender: %s" % f.getGender())
print("birthdate: %s" % f.getBirthdate())
print("patient_name: %s" % f.getPatientName())
print("patient_additional: %s" % f.getPatientAdditional())
print("admincode: %s" % f.getAdmincode())
print("technician: %s" % f.getTechnician())
print("equipment: %s" % f.getEquipment())
print("recording_additional: %s" % f.getRecordingAdditional())
print("datarecord duration: %f seconds" % f.getFileDuration())
print("number of datarecords in the file: %i" % f.datarecords_in_file)
print("number of annotations in the file: %i" % f.annotations_in_file)


library version: 0.1.15

general header:

edfsignals: 64
file duration: 61 seconds
startdate: 12-8-2009
starttime: 16:15:00
patientcode: 
gender: 
birthdate: 
patient_name: X
patient_additional: 
admincode: 
technician: 
equipment: BCI2000
recording_additional: 
datarecord duration: 61.000000 seconds
number of datarecords in the file: 61
number of annotations in the file: 1


In [5]:
print(sigbufs[0, :])
sigbufs[i, :].dtype

[33. 38.  3. ...  0.  0.  0.]


dtype('float64')

In [6]:
channel = 3
print("\nsignal parameters for the %d.channel:\n\n" % channel)

print("label: %s" % f.getLabel(channel))
print("samples in file: %i" % f.getNSamples()[channel])
# print("samples in datarecord: %i" % f.get
print("physical maximum: %f" % f.getPhysicalMaximum(channel))
print("physical minimum: %f" % f.getPhysicalMinimum(channel))
print("digital maximum: %i" % f.getDigitalMaximum(channel))
print("digital minimum: %i" % f.getDigitalMinimum(channel))
print("physical dimension: %s" % f.getPhysicalDimension(channel))
print("prefilter: %s" % f.getPrefilter(channel))
print("transducer: %s" % f.getTransducer(channel))
print("samplefrequency: %f" % f.getSampleFrequency(channel))

annotations = f.readAnnotations()
for n in np.arange(f.annotations_in_file):
    print("annotation: onset is %f    duration is %s    description is %s" % (annotations[0][n],annotations[1][n],annotations[2][n]))

buf = f.readSignal(channel)
n = 200
print("\nread %i samples\n" % n)
result = ""
for i in np.arange(n):
    result += ("%.1f, " % buf[i])
print(result)


signal parameters for the 3.channel:


label: Fcz.
samples in file: 9760
physical maximum: 8092.000000
physical minimum: -8092.000000
digital maximum: 8092
digital minimum: -8092
physical dimension: uV
prefilter: HP:0Hz LP:0Hz N:0Hz
transducer: BCI2000
samplefrequency: 160.000000
annotation: onset is 0.000000    duration is 60.2    description is T0

read 200 samples

13.0, 2.0, -19.0, 2.0, -10.0, -19.0, 1.0, -14.0, -18.0, -44.0, -30.0, -26.0, -25.0, -8.0, 0.0, -25.0, -24.0, -18.0, -21.0, 0.0, -28.0, -29.0, 14.0, 3.0, 7.0, 6.0, -1.0, 14.0, -4.0, -3.0, -1.0, -7.0, 3.0, 31.0, 5.0, -3.0, -20.0, -11.0, 20.0, 12.0, -12.0, -14.0, -5.0, 8.0, -11.0, -11.0, 13.0, -7.0, -16.0, -1.0, -7.0, -7.0, -6.0, 1.0, 46.0, 27.0, -3.0, -20.0, -16.0, -5.0, -18.0, 2.0, -1.0, -28.0, -21.0, -14.0, -4.0, 21.0, 41.0, 54.0, 47.0, 62.0, 68.0, 67.0, 51.0, 40.0, 40.0, 36.0, 28.0, -6.0, 10.0, 22.0, -8.0, -12.0, 6.0, 9.0, 10.0, 18.0, 0.0, 17.0, 26.0, 25.0, 36.0, 36.0, 19.0, -3.0, 5.0, 7.0, 20.0, 22.0, 1.0, 15.0, 8.0, -